In [ ]:
import pandas as pd
import numpy as np
from garmin_fit_sdk import Decoder, Stream
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import glob
from datetime import datetime
from scipy import signal
# from util import lowpass_filter
FIT_EPOCH_S = 631065600

In [ ]:
%pwd

In [ ]:
file_list = sorted(glob.glob("../../../data/virbs/all/*.fit"))
file_list

In [ ]:
# file_name = "2025-09-14-07-13-58"
# file_name = file_list[15]
file_name="../../../data/virbs/all/2025-11-02-07-11-57.fit"
stream = Stream.from_file(file_name)
decoder = Decoder(stream)
messages, errors = decoder.read(convert_datetimes_to_dates=False)
print(file_name)
errors

In [ ]:
starts = [m for m in messages['camera_event_mesgs'] if m['camera_event_type'] == 'video_start']
if len(starts) != 1:
    raise ValueError(f"Found {len(starts)} video start events")
start = starts[0]
start_time = start['timestamp'] * 1000 + start['timestamp_ms']
start_time

In [ ]:
gps_mesgs = messages['gps_metadata_mesgs']
gps_data = pd.DataFrame.from_records(gps_mesgs)
gps_data['utc_timestamp'] = pd.to_datetime((gps_data.utc_timestamp + FIT_EPOCH_S) * 1e9)
gps_data.index = (gps_data.timestamp * 1000 + gps_data.timestamp_ms) - start_time
gps_data.head()

In [ ]:
calibration_mesgs = messages['three_d_sensor_calibration_mesgs']
calibration_data = { m['sensor_type']: m for m in calibration_mesgs }
accel_cal = calibration_data['accelerometer']
accel_cal

In [ ]:
accel_raw_list = []
for group in messages['accelerometer_data_mesgs']:
    base_timestamp = group['timestamp'] * 1000 + group['timestamp_ms']
    for i, offset in enumerate(group['sample_time_offset']):
        entry = {
            'timestamp': base_timestamp + offset,
            'x': group['accel_x'][i],
            'y': group['accel_y'][i],
            'z': group['accel_z'][i],
        }
        accel_raw_list.append(entry)
accel_raw = pd.DataFrame.from_records(accel_raw_list)
accel_raw = accel_raw.set_index('timestamp').sort_index()
accel_raw.index = accel_raw.index - start_time

In [ ]:
accel_data = np.array(accel_cal['orientation_matrix']).reshape(3, 3) @ ((accel_raw.to_numpy() \
- accel_cal['level_shift'] - accel_cal['offset_cal']) * \
(accel_cal['calibration_factor'] / accel_cal['calibration_divisor'])).T
accel_data = accel_data.T
accel_data = pd.DataFrame(accel_data, columns=['x', 'y', 'z'], index=accel_raw.index)
fs = 1000 / np.mean(np.diff(accel_data.index))
accel_data['magnitude'] = np.sqrt(accel_data.x**2 + accel_data.y**2)


accel_filtered = pd.DataFrame(index=accel_data.index, data={
    'x': lowpass_filter(accel_data.x.to_numpy(), 5, fs),
    'y': lowpass_filter(accel_data.y.to_numpy(), 5, fs),
    'z': lowpass_filter(accel_data.z.to_numpy(), 5, fs),
})
accel_filtered['magnitude'] = np.sqrt(accel_filtered.x**2 + accel_filtered.y**2)

In [ ]:
# 4.986, 14.411
px.line(gps_data.enhanced_speed)

In [ ]:
px.line(accel_filtered.magnitude)

In [ ]:


fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=gps_data.index, y=gps_data.enhanced_speed, name="v"),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=accel_filtered.index, y=accel_filtered.magnitude, name="a"),
    secondary_y=True,
)
fig.show()


In [ ]:
f, Pxx = signal.welch(accel_filtered.magnitude.to_numpy(), fs, nperseg=1024)
px.line(pd.DataFrame(index=f, data=dict(PSD=Pxx)))

In [ ]:
import json
with open(f'../../../tmp/{file_name.split('/')[-1].replace(".fit", ".json")}', 'w') as f:
    json.dump(messages, f, indent=2, default=str)